In [ ]:
import sys
from pathlib import Path

# Jupyter Notebook 环境下的路径处理
try:
    # 尝试使用 __file__ (在脚本环境中)
    root_path = next(
        (
            p
            for p in Path(__file__).resolve().parents
            if (p / "pyproject.toml").is_file()
        ),
        None,
    )
except NameError:
    # 在 Jupyter Notebook 环境中，使用当前工作目录
    current_dir = Path.cwd()
    root_path = next(
        (
            p
            for p in [current_dir] + list(current_dir.parents)
            if (p / "pyproject.toml").is_file()
        ),
        None,
    )

if root_path:
    sys.path.insert(0, str(root_path))

In [ ]:
import time
from loguru import logger

from py_entry.data_conversion.backtest_runner import BacktestRunner
from py_entry.data_conversion.types import (
    BacktestParams,
    LogicOp,
    Param,
    SignalGroup,
    SignalTemplate,
    SettingContainer,
    ExecutionStage,
    PerformanceParams,
    PerformanceMetric,
)

from py_entry.data_conversion.data_generator import DataGenerationParams


# 创建 DataGenerationParams 对象
simulated_data_config = DataGenerationParams(
    timeframes=["15m", "1h", "4h"],
    start_time=1735689600000,
    num_bars=10000,
    fixed_seed=42,
    BaseDataKey="ohlcv_15m",
)

# 构建指标参数
indicators_params = {
    "ohlcv_15m": {
        "bbands": {
            "period": Param.create(14),
            "std": Param.create(2),
        }
    },
    "ohlcv_1h": {
        "rsi": {
            "period": Param.create(14),
        }
    },
    "ohlcv_4h": {
        "sma_0": {
            "period": Param.create(8),
        },
        "sma_1": {
            "period": Param.create(16),
        },
    },
}

# 自定义信号参数
signal_params = {
    # "rsi_upper": Param.create(70, 60, 80, 5),
    "rsi_center": Param.create(50, 40, 60, 5),
    # "rsi_lower": Param.create(30, 20, 40, 5),
}


# 自定义回测参数
backtest_params = BacktestParams(
    initial_capital=10000.0,
    fee_fixed=0,
    fee_pct=0.003,
    pause_drawdown=Param.create(0),
    pause_sma=Param.create(0),
    pause_ema=Param.create(0),
    exit_in_bar=True,
    use_extrema_for_exit=True,
    tsl_atr_tight=False,
    # sl_pct=Param.create(0.02),
    # tp_pct=Param.create(0.06),
    # tsl_pct=Param.create(0.02),
    sl_atr=Param.create(2),
    tp_atr=Param.create(6),
    tsl_atr=Param.create(2),
    atr_period=Param.create(14),
    tsl_psar_af0=Param.create(0.02),
    tsl_psar_af_step=Param.create(0.02),
    tsl_psar_max_af=Param.create(0.2),
)

# 自定义性能参数
performance_params = PerformanceParams(
    metrics=[
        PerformanceMetric.TOTAL_RETURN,
        PerformanceMetric.MAX_DRAWDOWN,
        PerformanceMetric.CALMAR_RATIO,
        PerformanceMetric.ANNUALIZATION_FACTOR,
        PerformanceMetric.HAS_LEADING_NAN_COUNT,
    ],
    leverage_safety_factor=0.8,
)

# 自定义信号模板
enter_long_group = SignalGroup(
    logic=LogicOp.AND,
    comparisons=[
        "close > bbands_upper",
        "rsi,ohlcv_1h, > $rsi_center",
        "sma_0,ohlcv_4h, > sma_1,ohlcv_4h,",
    ],
)

enter_short_group = SignalGroup(
    logic=LogicOp.AND,
    comparisons=[
        "close < bbands_lower",
        "rsi,ohlcv_1h, < $rsi_center",
        "sma_0,ohlcv_4h, < sma_1,ohlcv_4h,",
    ],
)

signal_template = SignalTemplate(
    enter_long=enter_long_group,
    enter_short=enter_short_group,
)

# 自定义引擎设置
engine_settings = SettingContainer(
    execution_stage=ExecutionStage.PERFORMANCE,
    return_only_final=False,
)


if __name__ == "__main__":
    # 配置logger

    start_time = time.perf_counter()

    # 创建启用时间测量的 BacktestRunner
    br = BacktestRunner(enable_timing=True)

    # 使用链式调用执行完整的回测流程
    logger.info("开始执行回测流程")

    # 完整的链式调用：配置 -> 运行 -> 添加索引 -> 保存 -> 上传

    # 完整的链式调用：配置 -> 运行 -> 添加索引 -> 保存 -> 上传
    br.setup(
        data_source=simulated_data_config,
        indicators_params=indicators_params,
        signal_params=signal_params,
        backtest_params=backtest_params,
        signal_template=signal_template,
        engine_settings=engine_settings,
    ).run().format_results_for_export(export_index=0, dataframe_format="parquet")

    if br.results:
        result = br.results[0]
        print(f"Performance: {result.performance}")

        logger.info(f"总耗时 {time.perf_counter() - start_time:.4f}秒")

In [ ]:
result

In [ ]:
from py_entry.data_conversion.file_utils import DisplayConfig, DashboardOverride


config = DisplayConfig(
    embed_data=False,
    width="100%",  # 图表容器的宽度，例如 "100%" 或 "800px"
    aspect_ratio="16/9",  # 图表容器的宽高比，例如 "16/9" 或 "4/3"
    override=DashboardOverride(
        show=["0,0,0,1"],
        showInLegend=["0,0,0,1"],
        showRiskLegend="1,1,1,1",
        showLegendInAll=True,
    ).to_dict(),
)

br.display_dashboard(config=config)